# PROFILING USING TENSORBOARD

## -> Computational Statistics

In [1]:
! pip install -U tensorboard_plugin_profile

In [2]:
# Import Depedencies
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import cv2
from tensorboard import notebook
import datetime
import tensorflow_datasets as tfds
from tensorflow.keras.layers import Input, Normalization, Conv2D, MaxPooling2D, Dense, Flatten, BatchNormalization, Dropout
from tensorflow.keras.metrics import BinaryAccuracy, FalsePositives, FalseNegatives, TrueNegatives, TruePositives, Precision, Recall, F1Score, AUC
from tensorflow.keras.regularizers import L2
from tensorboard.plugins.hparams import api as hp
import random

In [3]:
# Load Malaria Dataset From Tensorflow Datasets
dataset, datasetInfo = tfds.load("malaria", 
                                 with_info=True, 
                                 shuffle_files=True) # -----------++++++    ->    ---+++-++--+++-

# Dataset Split Function
def split(dataset, TRAIN_RATIO, VAL_RATIO, TEST_RATIO):
    trainDataset = dataset.take(int(TRAIN_RATIO*len(dataset)))
    leftDataset = dataset.skip(int(TRAIN_RATIO*len(dataset)))
    valDataset = leftDataset.take(int(VAL_RATIO*len(dataset)))
    testDataset = leftDataset.skip(int(VAL_RATIO*len(dataset)))
    return trainDataset, valDataset, testDataset

# Split Dataset As 80/10/10 
TRAIN_RATIO = 0.8
VAL_RATIO = 0.1
TEST_RATIO = 0.1
trainDataset, valDataset, testDataset = split(dataset['train'], TRAIN_RATIO, VAL_RATIO, TEST_RATIO)

IMAGE_SIZE = 224
def resizeRescale(inputs):
    return tf.image.resize(inputs['image'], (IMAGE_SIZE, IMAGE_SIZE))/255., inputs['label']
trainDataset = trainDataset.map(resizeRescale)
valDataset = valDataset.map(resizeRescale)
testDataset = testDataset.map(resizeRescale)

# Dataset Batching
BATCH_SIZE=32
BUFFER_SIZE = 8
trainDataset = trainDataset.shuffle(buffer_size=BUFFER_SIZE, reshuffle_each_iteration=True).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
valDataset = valDataset.shuffle(buffer_size=BUFFER_SIZE, reshuffle_each_iteration=True).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
testDataset = testDataset.shuffle(buffer_size=BUFFER_SIZE, reshuffle_each_iteration=True).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

2025-01-26 11:32:36.935118: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M3 Pro
2025-01-26 11:32:36.935144: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 18.00 GB
2025-01-26 11:32:36.935147: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 6.00 GB
2025-01-26 11:32:36.935160: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2025-01-26 11:32:36.935170: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [4]:
# Tensorboard Callback
CURRENT_TIME = datetime.datetime.now().strftime('%d%m%y - %h%m%s')
LOG_DIR = "./logs/" + CURRENT_TIME
tensorboardCallback = tf.keras.callbacks.TensorBoard(log_dir=LOG_DIR, histogram_freq=1, profile_batch='100, 132')

2025-01-26 11:32:37.024373: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:104] Profiler session initializing.
2025-01-26 11:32:37.024380: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:119] Profiler session started.
2025-01-26 11:32:37.024859: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:131] Profiler session tear down.


In [5]:
# Using Sequential API
IMAGE_SIZE = 224
model = tf.keras.Sequential([
    Input(shape=(IMAGE_SIZE, IMAGE_SIZE, 3)),

    Conv2D(filters=6, kernel_size=3, strides=1, padding='valid', activation='relu'),
    BatchNormalization(),
    MaxPooling2D(pool_size=2, strides=2),

    Conv2D(filters=16, kernel_size=3, strides=1, padding='valid', activation='relu'),
    BatchNormalization(),
    MaxPooling2D(pool_size=2, strides=2),

    Flatten(),

    Dense(100, activation='relu'),
    BatchNormalization(),
    Dense(10, activation='relu'),
    BatchNormalization(),
    Dense(1, activation='sigmoid'),
])
model.summary()

# Model Metrics
Metrics = [BinaryAccuracy(), FalsePositives(), FalseNegatives(), TrueNegatives(), TruePositives(), Precision(), Recall(), AUC()]

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),
    loss=tf.keras.losses.BinaryCrossentropy(),
    metrics=Metrics
)

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 222, 222, 6)    │           168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 222, 222, 6)    │            24 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 111, 111, 6)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 109, 109, 16)   │           880 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 109, 109, 16)   │            64 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 54, 54, 16)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 46656)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 100)            │     4,665,700 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 100)            │           400 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 10)             │         1,010 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 10)             │            40 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            11 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,668,297 (17.81 MB)

 Trainable params: 4,668,033 (17.81 MB)

 Non-trainable params: 264 (1.03 KB)

In [6]:
model.fit(trainDataset, validation_data=valDataset, epochs=5, verbose=1, callbacks=[tensorboardCallback])

Epoch 1/5


2025-01-26 11:32:37.933951: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


101/689 ━━━━━━━━━━━━━━━━━━━━ 42s 73ms/step - auc: 0.7054 - binary_accuracy: 0.6575 - false_negatives: 217.4653 - false_positives: 333.2574 - loss: 0.6611 - precision: 0.6510 - recall: 0.7325 - true_negatives: 453.3267 - true_positives: 627.9505

2025-01-26 11:32:47.247090: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:104] Profiler session initializing.
2025-01-26 11:32:47.247100: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:119] Profiler session started.


132/689 ━━━━━━━━━━━━━━━━━━━━ 40s 73ms/step - auc: 0.7096 - binary_accuracy: 0.6605 - false_negatives: 281.7651 - false_positives: 429.2576 - loss: 0.6537 - precision: 0.6517 - recall: 0.7345 - true_negatives: 606.7424 - true_positives: 810.2349

2025-01-26 11:32:49.568935: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:70] Profiler session collecting data.
2025-01-26 11:32:49.674351: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:131] Profiler session tear down.
2025-01-26 11:32:49.676433: I external/local_tsl/tsl/profiler/rpc/client/save_profile.cc:144] Collecting XSpace to repository: ./logs/260125 - Jan011737919957/train/plugins/profile/2025_01_26_11_32_49/Amans-MacBook-Pro.local.xplane.pb


689/689 ━━━━━━━━━━━━━━━━━━━━ 54s 75ms/step - auc: 0.7706 - binary_accuracy: 0.7130 - false_negatives: 1049.3319 - false_positives: 1867.8450 - loss: 0.5971 - precision: 0.6887 - recall: 0.7848 - true_negatives: 3650.8840 - true_positives: 4487.8867 - val_auc: 0.9025 - val_binary_accuracy: 0.5071 - val_false_negatives: 1339.0000 - val_false_positives: 19.0000 - val_loss: 1.0431 - val_precision: 0.7246 - val_recall: 0.0360 - val_true_negatives: 1347.0000 - val_true_positives: 50.0000
Epoch 2/5
689/689 ━━━━━━━━━━━━━━━━━━━━ 50s 72ms/step - auc: 0.9125 - binary_accuracy: 0.8578 - false_negatives: 317.6217 - false_positives: 1229.2406 - loss: 0.4352 - precision: 0.8075 - recall: 0.9421 - true_negatives: 4289.6650 - true_positives: 5219.4204 - val_auc: 0.9524 - val_binary_accuracy: 0.8610 - val_false_negatives: 74.0000 - val_false_positives: 309.0000 - val_loss: 0.3982 - val_precision: 0.8093 - val_recall: 0.9466 - val_true_negatives: 1061.0000 - val_true_positives: 1311.0000
Epoch 3/5
689/68

In [11]:
! tensorboard --logdir=logs

Serving TensorBoard on localhost; to expose to the network, use a proxy or pass --bind_all
TensorBoard 2.16.2 at http://localhost:6008/ (Press CTRL+C to quit)
W0126 11:41:39.526937 13517271040 security_validator.py:60] In 3.0, this warning will become an error:
Illegal Content-Security-Policy for script-src: 'unsafe-inline'
Illegal Content-Security-Policy for connect-src: data:
Illegal Content-Security-Policy for connect-src: www.gstatic.com
Illegal Content-Security-Policy for script-src-elem: 'unsafe-inline'
2025-01-26 11:41:40.923187: I tensorflow/core/profiler/convert/xplane_to_tools_data.cc:321] serving tool: tool_names with options: {}
2025-01-26 11:41:41.047442: I tensorflow/core/profiler/convert/xplane_to_tools_data.cc:321] serving tool: overview_page with options: {}
2025-01-26 11:43:08.449861: I tensorflow/core/profiler/convert/xplane_to_tools_data.cc:321] serving tool: framework_op_stats with options: {}
2025-01-26 11:43:11.161493: I tensorflow/core/profiler/convert/xplane_to